# Import necessary libraries



In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


Here, we import the required libraries:

os: Used for operating system dependent functionality (e.g., file paths).

numpy: Numerical computing library for handling arrays and matrices.

tensorflow and keras: Deep learning libraries for building and training neural networks.

ImageDataGenerator: Helps generate batches of tensor image data with real-time data augmentation.

Sequential, Conv2D, MaxPooling2D, Flatten, Dense: Components for building the CNN model.

# Define data directories and parameters

In [2]:
train_dir = 'train'# paths to the directories containing the training and validation images
validation_dir = 'train'

img_width, img_height = 150, 150
input_shape = (img_width, img_height, 3)
epochs = 10
batch_size = 32


Define the directories where the training and validation data are stored. Set the image dimensions, input shape, number of epochs for training, and batch size. 

# Data Preprocessing

In [9]:
train_datagen = ImageDataGenerator(rescale=1./255)

validation_datagen = ImageDataGenerator(rescale=1./255)
# rescale=1./255 is used to rescale the pixel values of the images to the range [0, 1].
#This normalization is performed by dividing each pixel value by 255.
#This step is important for neural networks as it helps in stabilizing the training process and improves convergence.


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),# dimensions to which all images will be resized during 
    batch_size=batch_size, # the number of images in each batch of data.
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')# tells type of label arrays that are returned


Found 557 images belonging to 2 classes.
Found 557 images belonging to 2 classes.


Data generators are created for both training and validation sets. These generators will load images from the specified directories, resize them to the target size, and normalize their pixel values. The flow_from_directory method generates batches of augmented data. In this case, since it's a binary classification problem (cats and dogs), class_mode is set to 'binary'.

#  Build CNN model

In [4]:
model = Sequential([ 
# initialize a sequential model, which is a linear stack of layers,layers are added one by one, and output of each layer feed into the next.
   
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    # adds 2D convolutional layer to the model(filters). 
    # 32: Number of filters (output channels).
    # (3, 3): Kernel size, specifying the height and width of the 2D convolution window.
    # activation='relu': Activation function applied element-wise to the output.
    # input_shape= Specifies the shape of the input data. This is the shape of the input images (height, width, channels).
    
    MaxPooling2D((2, 2)), 
    # specifies size of the pooling window or filter 
    # adds a max-pooling layer to the model, downsample feature maps while retaining the most important information.
    
    Conv2D(64, (3, 3), activation='relu'),
    # filters are increased from 32 to 64
    
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    #filters are increased from 64 to 128
    
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    # flattens input, means it converts 2D feature maps into a 1D vector, prepares data for input into the subsequent fully connected layers.
    
    Dense(512, activation='relu'),
# Dense: This is a fully connected layer in a neural network. 
# In dense layer, each neuron is connected to every neuron in the previous layer.
# 512: specifies no. of neurons in layer. Each neuron in the dense layer receives input from all neurons in previous layer and it produces a single output.
# activation='relu': activation function applied to  output of each neuron in the dense layer.
# the Rectified Linear Unit (ReLU) activation function is used, introduces non-linearity into the model, allowing it to learn complex patterns and relationships in the data. Mathematically, ReLU activation function is defined as ReLU activation function replaces all negative values with zero, while leaving positive values unchanged.
    
    Dense(1, activation='sigmoid')
])

# output layer with a single neuron and sigmoid activation function. For binary classification problems, sigmoid activation is commonly used to produce a probability output in the range [0, 1],
# where values closer to 1 represent one class (e.g., 'dog') and values closer to 0 represent the other class (e.g., 'cat').


A sequential model is created, representing a linear stack of layers. The model consists of several convolutional layers with max-pooling layers to extract features, followed by a flattening layer to convert the 2D feature maps into a 1D vector, and dense layers for classification. ReLU activation is used for all convolutional layers, and a sigmoid activation is used for the output layer for binary classification.

# Compile the model

In [5]:
model.compile(optimizer='adam',
# optimization algorithm to be used during the training of the neural network, refers to the Adam optimization algorithm
              loss='binary_crossentropy',
# measures difference b/w true labels and  predicted probabilities for binary classification tasks. Binary cross-entropy is for problems where output of the model is a probability distribution over two classes.              
              metrics=['accuracy'])
# measures the proportion of correctly classified samples among the total number of samples


The model is compiled with the Adam optimizer, binary cross-entropy loss function (suitable for binary classification), and accuracy metric for evaluation.


# Train the model

In [6]:
history = model.fit(
# fit() func. trains model on the provided training data and evaluates its performance on the validation data.
    train_generator,
 # train_generator yields batches of training samples during each iteration of training.
    
    steps_per_epoch=train_generator.samples // batch_size,
#  specifies the no. of steps to be taken during each epoch of training. It is calculated as the total no.of training samples divided by the batch size. This ensures that the model processes the entire training dataset in each epoch.
  
    epochs=epochs,
# epoch is one complete pass through entire training dataset. The model will iterate over the training data for the specified number of epochs, updating its weights based on the optimization algorithm and the loss function 
   
    validation_data=validation_generator,
# yields batches of validation samples for evaluating the model's performance after each epoch of training.
    
    validation_steps=validation_generator.samples // batch_size)
# tells the no.of steps to be taken during each epoch of validation


Epoch 1/10


17/17 [==============================] - 44s 2s/step - loss: 0.7242 - accuracy: 0.4762 - val_loss: 0.6947 - val_accuracy: 0.4908
Epoch 2/10
17/17 [==============================] - 39s 2s/step - loss: 0.6936 - accuracy: 0.5029 - val_loss: 0.6928 - val_accuracy: 0.5018
Epoch 3/10
17/17 [==============================] - 38s 2s/step - loss: 0.6928 - accuracy: 0.5048 - val_loss: 0.6905 - val_accuracy: 0.5607
Epoch 4/10
17/17 [==============================] - 41s 2s/step - loss: 0.6905 - accuracy: 0.5314 - val_loss: 0.6855 - val_accuracy: 0.5864
Epoch 5/10
17/17 [==============================] - 37s 2s/step - loss: 0.6993 - accuracy: 0.5486 - val_loss: 0.7453 - val_accuracy: 0.5018
Epoch 6/10
17/17 [==============================] - 35s 2s/step - loss: 0.6797 - accuracy: 0.5790 - val_loss: 0.6688 - val_accuracy: 0.6176
Epoch 7/10
17/17 [==============================] - 36s 2s/step - loss: 0.6656 - accuracy: 0.5657 - val_loss: 0.6515 - val_accuracy: 0.6121
Epoch 8/10
17/17 [

The model is trained using the fit method. It takes the training generator, steps per epoch, number of epochs, validation generator, and validation steps as arguments.

# Evaluate the model

In [7]:
loss, accuracy = model.evaluate(validation_generator)
# variable stores the value of the loss function, loss value indicates how well model is performing on the validation data, with lower values indicating better performance.
# After evaluation, variable stores the value of the accuracy metric computed on the validation dataset

print("Validation Accuracy: {:.2f}%".format(accuracy * 100))
# {:.2f}% is a placeholder for a floating-point number with two decimal places, % sign indicates that the accuracy will be displayed as a percentage , multiplies the accuracy value by 100 to convert it to a percentage.

18/18 [==============================] - 15s 848ms/step - loss: 0.5927 - accuracy: 0.6607
Validation Accuracy: 66.07%


The trained model is evaluated on the validation set, and the validation accuracy is printed.

In summary, the provided code can be used to train a CNN for the classification of images into two categories: dogs and cats. After training, the model can predict the probability of an input images belonging to each class, allowing it to classify new images accordingly if added.


